# Advent of Code 2024 Day 20 

### Setup

In [123]:
from aocd import get_data, submit

day = 20
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

In [ ]:
import numpy as np

def parse_data(raw_data):
    lines = raw_data.splitlines()
    
    data = [list(line) for line in lines]
    
    return np.array(data)

sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data[0]

### Part One!

In [202]:
use_sample_data = False
part = 'a'

In [203]:
data = sample_data if use_sample_data else test_data

In [204]:
from typing import Literal, Union

Track = Literal['.']
Wall = Literal['#']
Start = Literal['S']
End = Literal['E']

Tile = Union[Track, Wall, Start, End]

In [205]:
from functools import total_ordering

@total_ordering
class KeyDict(object):
    def __init__(self, key, dct):
        self.key = key
        self.dct = dct

    def __lt__(self, other):
        return self.key < other.key

    def __eq__(self, other):
        return self.key == other.key

    def __repr__(self):
        return '{0.__class__.__name__}(key={0.key}, dct={0.dct})'.format(self)
    
    def __iter__(self):
        return iter((self.key, self.dct))
    
    def __getitem__(self, key):
        return self.dct[key]
    
    def __setitem__(self, key, value):
        self.dct[key] = value

In [215]:
def get_candidate_positions(arr:np.ndarray, pos:np.ndarray, remove_walls:bool=True):
    directions = [
        [0, 1],
        [0, -1],
        [1, 0],
        [-1, 0]
    ]
    
    for dir in directions:
        candidate_pos = pos + dir
        if remove_walls and arr[tuple(candidate_pos)] == '#':
            continue
        elif candidate_pos[0] < 0 or candidate_pos[1] < 0:
            continue
        elif candidate_pos[0] >= arr.shape[0] or candidate_pos[1] >= arr.shape[1]:
            continue
        
        yield np.array(candidate_pos)

In [216]:
def euclidean_distance(a:np.ndarray, b:np.ndarray):
    return np.sqrt(np.sum((a - b)**2))

def manhattan_distance(a:np.ndarray, b:np.ndarray):
    return np.sum(np.abs(a - b))

def dummy_heuristic(a:np.ndarray, b:np.ndarray):
    return 0

In [217]:
def cost (arr:np.ndarray, a:np.ndarray, b:np.ndarray):
    return 1

In [218]:
import heapq

def astar(arr:np.ndarray, start:np.ndarray, end:np.ndarray, heuristic_fn = manhattan_distance, cost_fn = cost):
    queue = [KeyDict(0, { 'path': [start] })]
    visited = set()
    
    while queue:
        score, current = heapq.heappop(queue)
        current_pos = current['path'][-1]
        current_path = current['path']
        
        if current_pos[0] == end[0] and current_pos[1] == end[1]:
            return current_path
        
        if tuple(current_pos) in visited:
            continue
        
        visited.add(tuple(current_pos))
        
        for candidate_pos in get_candidate_positions(arr, current_pos):
            candidate_path = current_path[:] + [ candidate_pos ]
            candidate_cost = score + cost_fn(arr, current_pos, candidate_pos) + heuristic_fn(candidate_pos, end)
            
            heapq.heappush(queue, KeyDict(candidate_cost, { 'path': candidate_path }))

In [219]:
start = np.argwhere(data == 'S')[0]
end = np.argwhere(data == 'E')[0]

baseline_path = astar(data, start, end)
baseline_node_distance = { tuple(node): (len(baseline_path) - 1) - i for i, node in enumerate(baseline_path) }

baseline_data = {
    'path': baseline_path,
    'node_distance': baseline_node_distance
}

baseline_data

{'path': [array([75, 65]),
  array([75, 64]),
  array([75, 63]),
  array([75, 62]),
  array([75, 61]),
  array([75, 60]),
  array([75, 59]),
  array([74, 59]),
  array([73, 59]),
  array([72, 59]),
  array([71, 59]),
  array([71, 58]),
  array([71, 57]),
  array([72, 57]),
  array([73, 57]),
  array([73, 56]),
  array([73, 55]),
  array([72, 55]),
  array([71, 55]),
  array([71, 54]),
  array([71, 53]),
  array([72, 53]),
  array([73, 53]),
  array([73, 52]),
  array([73, 51]),
  array([72, 51]),
  array([71, 51]),
  array([71, 50]),
  array([71, 49]),
  array([72, 49]),
  array([73, 49]),
  array([73, 48]),
  array([73, 47]),
  array([73, 46]),
  array([73, 45]),
  array([73, 44]),
  array([73, 43]),
  array([73, 42]),
  array([73, 41]),
  array([74, 41]),
  array([75, 41]),
  array([75, 40]),
  array([75, 39]),
  array([74, 39]),
  array([73, 39]),
  array([72, 39]),
  array([71, 39]),
  array([71, 38]),
  array([71, 37]),
  array([72, 37]),
  array([73, 37]),
  array([74, 37]),
  ar

In [220]:
one_step_directions = np.array([
    [0, 1],
    [1, 0],
    [0, -1],
    [-1, 0]
])

two_step_directions = np.array([
    [0, 2],
    [2, 0],
    [0, -2],
    [-2, 0]
])

path = baseline_data['path'].copy()
speedups = {}
for idx in path:
    one_steps = one_step_directions + idx
    
    rows = one_steps[:, 0]
    cols = one_steps[:, 1]
    
    mask = data[rows, cols] == '#'
    two_steps = two_step_directions[mask] + idx

    for next_idx in [ i for i in two_steps if tuple(i) in baseline_data['node_distance']]:
        original_speed = baseline_data['node_distance'][tuple(idx)]
        new_speed = baseline_data['node_distance'][tuple(next_idx)] + 2 # +2 to account for the two steps
        
        speedup = original_speed - new_speed
        
        if speedup >= 0:
            speedups.setdefault(speedup, 0)
            speedups[speedup] += 1
    
speedups

{2716: 2,
 2712: 1,
 2710: 1,
 2708: 1,
 2706: 1,
 2704: 1,
 2700: 2,
 4: 1007,
 2698: 1,
 2: 1007,
 2696: 2,
 2608: 1,
 200: 14,
 192: 11,
 190: 6,
 188: 14,
 180: 12,
 178: 6,
 172: 13,
 176: 13,
 168: 12,
 166: 7,
 164: 18,
 152: 21,
 150: 11,
 148: 18,
 8: 449,
 6: 274,
 140: 11,
 132: 15,
 130: 5,
 96: 29,
 128: 12,
 94: 18,
 92: 34,
 9356: 1,
 9354: 1,
 9352: 1,
 88: 35,
 86: 15,
 84: 39,
 9348: 1,
 9344: 1,
 9342: 1,
 9340: 1,
 76: 33,
 74: 15,
 68: 44,
 72: 41,
 9332: 1,
 60: 39,
 64: 44,
 9328: 1,
 9326: 1,
 9324: 1,
 9308: 1,
 9306: 1,
 9304: 1,
 52: 45,
 50: 22,
 44: 64,
 48: 65,
 9296: 1,
 40: 79,
 9292: 1,
 9244: 1,
 9242: 1,
 9240: 1,
 32: 99,
 30: 47,
 28: 109,
 9232: 1,
 9230: 1,
 9228: 1,
 9220: 1,
 16: 233,
 14: 149,
 9216: 1,
 12: 345,
 9214: 1,
 9212: 1,
 9204: 1,
 340: 8,
 304: 6,
 296: 4,
 280: 8,
 268: 7,
 228: 5,
 224: 7,
 220: 13,
 10: 214,
 24: 147,
 22: 82,
 20: 180,
 198: 7,
 196: 16,
 186: 3,
 184: 11,
 174: 7,
 62: 21,
 36: 81,
 34: 35,
 18: 90,
 2404: 1,


In [221]:
target = 100

part_a_answer = sum([ speedups[key] for key in speedups.keys() if key >= target ])
part_a_answer

1389

In [222]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

aocd will not submit that answer again. At 2024-12-23 10:17:02.654629-05:00 you've previously submitted 1389 and the server responded with:
That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


### Part Two!

In [ ]:
use_sample_data = True
part='b'

In [ ]:
data = sample_data if use_sample_data else test_data

In [ ]:
part_b_answer = 0

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)